In [ ]:
import os
import re

config="HPP"
apps=["dijkstra","qsort","string_search","gsm","jpeg"]

# get time 
def get_time(file):
    f = open(file)
    lines = f.readlines()
    for line in lines:
        if line.find("Time (ns)")>0:
            return int(re.findall(r'\d+', line)[0])
    return -1

def get_cpi(file):
    f = open(file)
    lines = f.readlines()
    for line in lines:
        if line.find("  IPC")>=0:
            return float(re.findall(r'\d+\.\d+', line)[0])
    return -1

def get_power(file):
    area=0.0
    peak_dynamic=0.0
    runtime_dynamic=0.0
    leakage=0.0
    
    f = open(file)
    lines = f.readlines()
    for line in lines[0:20]:
        if line.find("  Area =")>=0:
            area=float(re.findall(r'\d+\.\d+', line)[0])
            # print("area", area)
        if line.find("  Peak Dynamic =")>=0:
            peak_dynamic=float(re.findall(r'\d+\.\d+', line)[0])
            # print("peak_dynamic", peak_dynamic)     
        if line.find("  Runtime Dynamic =")>=0:
            runtime_dynamic=float(re.findall(r'\d+\.\d+', line)[0])
            # print("runtime_dynamic",runtime_dynamic)             
        if line.find("  Subthreshold Leakage =")>=0:
            leakage=float(re.findall(r'\d+\.\d+', line)[0])
            # print("leakage",leakage)               
    return area,peak_dynamic,runtime_dynamic,leakage        


In [50]:

# config="HPP-PW-S/151-LSQ-08-RS-32-WS-064-DP-08-CM-01"
config="HPP-PW-S/225-LSQ-16-RS-32-WS-016-DP-08-CM-08"
for app in apps:
    print(app)
    time = get_time("/".join([config,app,"sim.out"]))
    print("time:", time)

    print("CPI:", 1/get_cpi("/".join([config,app,"sim.out"]))) 
    
    infos = get_power("/".join([config,app,"power.txt"]))
    print("area\t Pd\t Rd\t Lk")
    print(infos)
    print("energy:%.6f"% ((infos[2]+infos[3])* time / 1e9))
    print("EDP:%.6f"% ((infos[2]+infos[3])* time / 1e9 * (time / 1e9 )))



dijkstra
time: 59860700
CPI: 1.5384615384615383
area	 Pd	 Rd	 Lk
(13.9793, 13.0158, 0.0, 0.716799)
energy:0.042908
EDP:0.002569
qsort
time: 54928400
CPI: 7.142857142857142
area	 Pd	 Rd	 Lk
(13.9793, 12.2975, 0.0, 0.716799)
energy:0.039373
EDP:0.002163
string_search
time: 1476600
CPI: 7.692307692307692
area	 Pd	 Rd	 Lk
(13.9793, 12.2754, 0.0, 0.716799)
energy:0.001058
EDP:0.000002
gsm
time: 6922500
CPI: 1.7241379310344829
area	 Pd	 Rd	 Lk
(13.9793, 12.7548, 0.0, 0.716799)
energy:0.004962
EDP:0.000034
jpeg
time: 24168000
CPI: 2.0833333333333335
area	 Pd	 Rd	 Lk
(13.9793, 12.7619, 0.0, 0.716799)
energy:0.017324
EDP:0.000419


In [ ]:
for app in apps:
    print(app)
    base_info = get_power("/".join(["base",app,"power.txt"]))
    hpp_info = get_power("/".join(["HPP",app,"power.txt"]))
    area_inc = (hpp_info[0]-base_info[0])/base_info[0] * 100
    
    base_time =  get_time("/".join(["base",app,"sim.out"]))
    hpp_time =  get_time("/".join(["HPP",app,"sim.out"]))
    
    
    base_energy = (base_info[2]+base_info[3])* base_time / 1e9
    hpp_energy = (hpp_info[2]+hpp_info[3])* hpp_time / 1e9
    energy_inc = (hpp_energy-base_energy)/base_energy * 100
    print("area inc %.4f%%"%area_inc)
    print("energy inc %.4f%%"%energy_inc)

In [ ]:
import matplotlib.pyplot as plt
import squarify  # 

# Peak Dynamic
labels = ["Instruction Fetch Unit", "Renaming Unit", "Load Store Unit","Memory Management Unit" ,"Execution Unit", "L2"]
peak_dynamic_values = [2.80978, 3.26111, 0.477192,0.184631, 5.42389, 0.0332925]  # 

# 
plt.figure(figsize=(8, 6))
squarify.plot(sizes=peak_dynamic_values, label=labels, alpha=0.7)
# 
plt.axis('off')  # 
plt.title('Peak Dynamic Distribution')

# 
plt.tight_layout()
plt.show()

In [46]:
# get CPI

LSQ_list = [1,8,16]
RS_list = [1,16,32]
WS_list = [16,64,128]
DP_list = [1,4,8]
CM_list = [1,4,8]

index = 0

plist = []
print("config,Peak dynamic,CPI,energy,EDP")
for LSQ in LSQ_list:
    for RS in RS_list:
        for WS in WS_list:
            for DP in DP_list:
                for CM in CM_list:
                    try:
                        index +=1
                        f = "%.3d_LSQ_%.2d_RS_%.2d_WS_%.3d_DP_%.2d_CM_%.2d"%(index,LSQ,RS,WS,DP,CM)
                        ff = os.path.join("HPP-PW-S",f)
                        time = get_time("/".join([ff,"dijkstra","sim.out"]))
                        
                        cpi = 1.0/ get_cpi("/".join([ff,"dijkstra","sim.out"]))
                        if(cpi==0):
                            continue
                        infos = get_power("/".join([ff,"dijkstra","power.txt"]))
                        if(infos[1] == 0):
                            continue
                        if(infos[1]<8):
                            continue
                        # if(infos[1]>11.5):
                        #     continue
                        e = (infos[2]+infos[3])* time / 1e9
                        edp = e * (time / 1e9 )
                        print("%s,%.6f,%.6f,%.6f,%.6f"%(f,infos[1],cpi,e,edp ))
                    except Exception as e:
                        pass

config,Peak dynamic,CPI,energy,EDP
034_LSQ_01_RS_16_WS_016_DP_08_CM_01,12.676600,2.439024,0.067049,0.006272
035_LSQ_01_RS_16_WS_016_DP_08_CM_04,12.682800,2.380952,0.066363,0.006144
036_LSQ_01_RS_16_WS_016_DP_08_CM_08,12.683600,2.380952,0.066255,0.006124
043_LSQ_01_RS_16_WS_064_DP_08_CM_01,11.131100,2.325581,0.177860,0.016205
044_LSQ_01_RS_16_WS_064_DP_08_CM_04,11.132200,2.325581,0.177666,0.016157
045_LSQ_01_RS_16_WS_064_DP_08_CM_08,11.132100,2.325581,0.177618,0.016147
061_LSQ_01_RS_32_WS_016_DP_08_CM_01,12.676300,2.439024,0.067115,0.006284
062_LSQ_01_RS_32_WS_016_DP_08_CM_04,12.682400,2.380952,0.066406,0.006152
063_LSQ_01_RS_32_WS_016_DP_08_CM_08,12.682400,2.380952,0.066403,0.006151
070_LSQ_01_RS_32_WS_064_DP_08_CM_01,11.135900,2.325581,0.177025,0.016000
071_LSQ_01_RS_32_WS_064_DP_08_CM_04,11.135900,2.325581,0.177053,0.016006
072_LSQ_01_RS_32_WS_064_DP_08_CM_08,11.135500,2.325581,0.177114,0.016022
088_LSQ_08_RS_01_WS_016_DP_08_CM_01,12.484900,3.571429,0.098281,0.013475
089_LSQ_08_RS_01

In [ ]:
# get CPI for all app with HPP-PW
HPP_PW_list=[
    "124-LSQ-08-RS-16-WS-064-DP-08-CM-01",
    "205-LSQ-16-RS-16-WS-064-DP-08-CM-01",
    "151-LSQ-08-RS-32-WS-064-DP-08-CM-01",
    "232-LSQ-16-RS-32-WS-064-DP-08-CM-01",
    "043-LSQ-01-RS-16-WS-064-DP-08-CM-01",
    "045-LSQ-01-RS-16-WS-064-DP-08-CM-08",
    "044-LSQ-01-RS-16-WS-064-DP-08-CM-04",
    "072-LSQ-01-RS-32-WS-064-DP-08-CM-08",
    "070-LSQ-01-RS-32-WS-064-DP-08-CM-01",
    "071-LSQ-01-RS-32-WS-064-DP-08-CM-04"
]

apps=["dijkstra","qsort","string_search","gsm","jpeg"]

print("config,metric,dijkstra,qsort,string_search,gsm,jpeg")
for f in HPP_PW_list:
    pws = []
    cpis = []
    for app in apps:
        ff = os.path.join("HPP-PW-S",f)
        cpi = 1.0/ get_cpi("/".join([ff,app,"sim.out"]))
        if(cpi==0):
            continue
        infos = get_power("/".join([ff,app,"power.txt"]))
        pws.append(infos[1])
        cpis.append(cpi)
    print("%s,Peak dynamic,%.6f,%.6f,%.6f,%.6f,%.6f"%(f,pws[0],pws[1],pws[2],pws[3],pws[4]))
    print("%s,CPI,%.6f,%.6f,%.6f,%.6f,%.6f"%(f,cpis[0],cpis[1],cpis[2],cpis[3],cpis[4]))
        
